In [ ]:
import numpy as np
import matplotlib.pyplot as plt # version > 3.0
import pandas as pd
from datetime import datetime
from google.colab import drive
import math
plt.style.use('seaborn')

In [ ]:
path_to_sepsis3_single_stay_csv = "/content/Sepsis_Labs_Vitals_Demographic.csv"
sepsis_original = pd.read_csv(path_to_sepsis3_single_stay_csv)
path_to_sepsis3_many_stays_csv = "/content/sepsis_labs_avg.csv"
path_to_vitals_stays_csv = "/content/vitals_avg.csv"
sepsis_many_stays_df = pd.read_csv(path_to_sepsis3_many_stays_csv)
vitals_df = pd.read_csv(path_to_vitals_stays_csv)
print(sepsis_original.shape)
print(sepsis_many_stays_df.shape)
print(vitals_df.shape)
sepsis_original.info()
sepsis_original.describe()
sepsis_many_stays_df.describe()
sepsis_many_stays_df.info()

vitals_list = []

for column, _ in vitals_df.iteritems():
  vitals_list.append(column)

print(vitals_list)

sepsis_lab_list = []

for column, _ in sepsis_many_stays_df.iteritems():
  sepsis_lab_list.append(column)

print(sepsis_lab_list)

def find_first_non_null_sepsis_lab(subject_id, stay_id, column_name):
  modified_sepsis_lab_1 = sepsis_many_stays_df[sepsis_many_stays_df["subject_id"] == subject_id]
  modified_sepsis_lab_2 = modified_sepsis_lab_1[modified_sepsis_lab_1["stay_id"] == stay_id]
  for _, row in modified_sepsis_lab_2.iterrows():
    if type(row[column_name+"_avg"]) != str and not math.isnan(row[column_name+"_avg"]):
      return row[column_name+"_avg"]
    else:
      return None

def find_first_non_null_vital(subject_id, stay_id, column_name):
  modified_vitals_1 = vitals_df[vitals_df["subject_id"] == subject_id]
  modified_vitals_2 = modified_vitals_1[modified_vitals_1["stay_id"] == stay_id]
  for _, row in modified_vitals_2.iterrows():
    if type(row[column_name+"_avg"]) != str and not math.isnan(row[column_name+"_avg"]):
      return row[column_name+"_avg"]
    else:
      return None

sepsis_non_null_df = sepsis_original.copy(deep=True)

for column,_ in sepsis_original.iteritems():
  for i,row in sepsis_original.iterrows():
    if column+"_avg" in sepsis_lab_list and math.isnan(row[column]):
      non_null_value = find_first_non_null_sepsis_lab(row["subject_id"], row["stay_id"], column)
      if non_null_value is not None:
        print("Supposed to be non null: ",non_null_value)
      sepsis_non_null_df.at[i, column] = non_null_value
    elif column+"_avg" in vitals_list and math.isnan(row[column]):
      non_null_value = find_first_non_null_vital(row["subject_id"], row["stay_id"], column)
      if non_null_value is not None:
        print("Supposed to be non null: ",non_null_value)
      sepsis_non_null_df.at[i, column] = non_null_value
    else:
      continue

sepsis_non_null_df.info()
sepsis_non_null_df.to_csv("/content/Sepsis_Lab_and_Vital_with_Stay_Avg.csv")
  


In [ ]:
path_to_csv = "/content/Sepsis_Lab_and_Vital_with_Stay_Avg.csv"
updated_df = pd.read_csv(path_to_csv)
updated_df_copy = updated_df.copy(deep=True)
updated_df_copy_1 = updated_df_copy.fillna(value={"pao2fio2ratio_novent": float(np.random.normal(450,20,1)),
                                                  "pao2fio2ratio_vent": float(np.random.normal(440,20,1)),
                                                  "rate_dobutamine":0 ,
                                                  "rate_epinephrine":0,
                                                  "rate_norepinephrine":0, 
                                                  "rate_dopamine":0,
                                                  "meanbp_min":float(np.random.normal(100,10,1) ), 
                                                  "gcs_min":float(np.random.normal(16,2,1)), 
                                                  "uo_24hr": float(np.random.normal(550,20,1))  ,
                                                  "bilirubin_max":float(np.random.normal(0.8,0.1,1)) ,
                                                  "creatinine_max":float(np.random.normal(0.8,0.1,1)),
                                                  "platelet_min":float(np.random.normal(180,10,1)),
                                                  "heart_rate":float(np.random.normal(80,10,1)), 
                                                  "sbp":float(np.random.normal(120,10,1)), 
                                                  "mbp":float(np.random.normal(100,10,1)), 
                                                  "dbp":float(np.random.normal(80,10,1)), 
                                                  "sbp_ni":float(np.random.normal(120,10,1)), 
                                                  "mbp_ni":float(np.random.normal(100,15,1)),
                                                  "dbp_ni":float(np.random.normal(80,5,1)), 
                                                  "temperature":float(np.random.normal(97,3,1)),
                                                  "glucose":float(np.random.normal(140,10,1)),
                                                  "spo2":float(np.random.normal(95,2,1)),
                                                  "resp_rate":float(np.random.normal(14,1,1)) })

updated_df_copy_1.describe()
updated_df_copy_1.to_csv("/content/No_NaN_Values_New.csv")
